In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
# import joblib

# # Load the dataset
# dataset_path = "flood_prediction_dataset.csv"
# data = pd.read_csv(dataset_path)

# # Convert date to datetime
# data['date'] = pd.to_datetime(data['date'])

# # Exploratory Data Analysis
# def perform_eda(data):
#     print("Dataset shape:", data.shape)
#     print("\nData types:")
#     print(data.dtypes)
    
#     print("\nCheck for missing values:")
#     print(data.isnull().sum())
    
#     # Correlation matrix for numerical features
#     plt.figure(figsize=(14, 10))
#     numeric_data = data.select_dtypes(include=[np.number])
#     correlation = numeric_data.corr()
#     sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5)
#     plt.title("Correlation Matrix")
#     plt.savefig("correlation_matrix.png")
#     plt.close()
    
#     # Distribution of flood events by region
#     plt.figure(figsize=(10, 6))
#     flood_by_region = data.groupby('region')['flood_occurred'].mean() * 100
#     flood_by_region.sort_values().plot(kind='barh')
#     plt.title("Percentage of Flood Events by Region")
#     plt.xlabel("Flood Occurrence (%)")
#     plt.savefig("flood_by_region.png")
#     plt.close()
    
#     # Rainfall vs Flood Occurrence
#     plt.figure(figsize=(12, 6))
#     sns.boxplot(x='flood_occurred', y='rainfall_72h', data=data)
#     plt.title("72-Hour Rainfall vs Flood Occurrence")
#     plt.savefig("rainfall_vs_flood.png")
#     plt.close()
    
#     # Seasonal patterns
#     plt.figure(figsize=(12, 6))
#     monthly_floods = data.groupby('month')['flood_occurred'].mean() * 100
#     monthly_floods.plot(kind='bar')
#     plt.title("Monthly Flood Occurrence Percentage")
#     plt.xlabel("Month")
#     plt.ylabel("Flood Occurrence (%)")
#     plt.savefig("monthly_floods.png")
#     plt.close()
    
#     # Feature distributions based on flood occurrence
#     important_features = ['rainfall_24h', 'rainfall_72h', 'river_level', 'soil_moisture', 'reservoir_level']
#     fig, axes = plt.subplots(len(important_features), 1, figsize=(12, 15))
    
#     for i, feature in enumerate(important_features):
#         sns.kdeplot(data=data, x=feature, hue='flood_occurred', ax=axes[i])
#         axes[i].set_title(f'{feature} Distribution by Flood Occurrence')
    
#     plt.tight_layout()
#     plt.savefig("feature_distributions.png")
#     plt.close()

# # Preprocess data
# def preprocess_data(data):
#     # Separate features and target
#     X = data.drop(['flood_occurred', 'flood_probability', 'date'], axis=1)
#     y = data['flood_occurred']
    
#     # Split categorical and numerical columns
#     categorical_cols = ['region']
#     numerical_cols = [col for col in X.columns if col not in categorical_cols]
    
#     # Create preprocessing steps
#     numerical_transformer = StandardScaler()
#     categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
#     # Create column transformer
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numerical_transformer, numerical_cols),
#             ('cat', categorical_transformer, categorical_cols)
#         ])
    
#     # Split the data
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
#     return X_train, X_test, y_train, y_test, preprocessor, numerical_cols, categorical_cols

# def build_models(X_train, y_train, preprocessor):
#     # Model 1: Random Forest
#     rf_model = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', RandomForestClassifier(random_state=42))
#     ])
    
#     # Model 2: Gradient Boosting
#     gb_model = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', GradientBoostingClassifier(random_state=42))
#     ])
    
#     # Model 3: XGBoost
#     xgb_model = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
#     ])
    
#     # Model 4: Logistic Regression (baseline)
#     lr_model = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', LogisticRegression(random_state=42, max_iter=1000))
#     ])
    
#     return {
#         'Random Forest': rf_model,
#         'Gradient Boosting': gb_model,
#         'XGBoost': xgb_model,
#         'Logistic Regression': lr_model
#     }

# def evaluate_models(models, X_train, y_train, X_test, y_test):
#     results = {}
    
#     for name, model in models.items():
#         print(f"\nTraining {name}...")
#         model.fit(X_train, y_train)
        
#         # Cross-validation
#         cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
#         print(f"{name} Cross-Validation ROC-AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
#         # Predictions on test set
#         y_pred = model.predict(X_test)
#         y_pred_proba = model.predict_proba(X_test)[:, 1]
        
#         # Classification report
#         print(f"\n{name} Classification Report:")
#         print(classification_report(y_test, y_pred))
        
#         # Confusion matrix
#         cm = confusion_matrix(y_test, y_pred)
#         plt.figure(figsize=(8, 6))
#         sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
#         plt.title(f'{name} Confusion Matrix')
#         plt.ylabel('True Label')
#         plt.xlabel('Predicted Label')
#         plt.savefig(f"{name.replace(' ', '_').lower()}_confusion_matrix.png")
#         plt.close()
        
#         # ROC curve
#         fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
#         roc_auc = auc(fpr, tpr)
        
#         plt.figure(figsize=(8, 6))
#         plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.3f})')
#         plt.plot([0, 1], [0, 1], 'k--')
#         plt.xlim([0.0, 1.0])
#         plt.ylim([0.0, 1.05])
#         plt.xlabel('False Positive Rate')
#         plt.ylabel('True Positive Rate')
#         plt.title(f'{name} ROC Curve')
#         plt.legend(loc="lower right")
#         plt.savefig(f"{name.replace(' ', '_').lower()}_roc_curve.png")
#         plt.close()
        
#         # Precision-Recall curve
#         precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
#         pr_auc = auc(recall, precision)
        
#         plt.figure(figsize=(8, 6))
#         plt.plot(recall, precision, label=f'PR curve (area = {pr_auc:.3f})')
#         plt.xlabel('Recall')
#         plt.ylabel('Precision')
#         plt.title(f'{name} Precision-Recall Curve')
#         plt.legend(loc="lower left")
#         plt.savefig(f"{name.replace(' ', '_').lower()}_pr_curve.png")
#         plt.close()
        
#         results[name] = {
#             'model': model,
#             'cv_score': cv_scores.mean(),
#             'roc_auc': roc_auc,
#             'pr_auc': pr_auc
#         }
    
#     return results

# def fine_tune_best_model(best_model_name, models, X_train, y_train, X_test, y_test):
#     print(f"\nFine-tuning {best_model_name}...")
    
#     if best_model_name == 'Random Forest':
#         param_grid = {
#             'classifier__n_estimators': [100, 200, 300],
#             'classifier__max_depth': [None, 10, 20, 30],
#             'classifier__min_samples_split': [2, 5, 10],
#             'classifier__min_samples_leaf': [1, 2, 4]
#         }
#     elif best_model_name == 'Gradient Boosting':
#         param_grid = {
#             'classifier__n_estimators': [100, 200, 300],
#             'classifier__learning_rate': [0.01, 0.1, 0.2],
#             'classifier__max_depth': [3, 5, 7]
#         }
#     elif best_model_name == 'XGBoost':
#         param_grid = {
#             'classifier__n_estimators': [100, 200, 300],
#             'classifier__learning_rate': [0.01, 0.1, 0.2],
#             'classifier__max_depth': [3, 5, 7],
#             'classifier__subsample': [0.8, 0.9, 1.0]
#         }
#     else:  # Logistic Regression
#         param_grid = {
#             'classifier__C': [0.01, 0.1, 1, 10, 100],
#             'classifier__penalty': ['l2'],
#             'classifier__solver': ['lbfgs', 'liblinear']
#         }
    
#     grid_search = GridSearchCV(
#         models[best_model_name],
#         param_grid,
#         cv=5,
#         scoring='roc_auc',
#         n_jobs=-1
#     )
    
#     grid_search.fit(X_train, y_train)
    
#     print(f"Best parameters: {grid_search.best_params_}")
#     print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
#     # Evaluate the tuned model
#     best_model = grid_search.best_estimator_
#     y_pred = best_model.predict(X_test)
#     y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
#     print("\nTuned Model Classification Report:")
#     print(classification_report(y_test, y_pred))
    
#     # Save the tuned model
#     joblib.dump(best_model, "flood_prediction_model.pkl")
    
#     return best_model

# def feature_importance(best_model, numerical_cols, categorical_cols):
#     if hasattr(best_model.named_steps['classifier'], 'feature_importances_'):
#         # Get feature names from the preprocessor
#         preprocessor = best_model.named_steps['preprocessor']
#         feature_names = []
        
#         # Add numerical feature names
#         feature_names.extend(numerical_cols)
        
#         # Add one-hot encoded categorical feature names
#         onehot_features = []
#         for cat in categorical_cols:
#             encoder = preprocessor.named_transformers_['cat']
#             categories = encoder.categories_[0]
#             onehot_features.extend([f"{cat}_{c}" for c in categories])
        
#         feature_names.extend(onehot_features)
        
#         # Get feature importances
#         importances = best_model.named_steps['classifier'].feature_importances_
        
#         # If the lengths don't match, adjust feature names
#         if len(importances) != len(feature_names):
#             feature_names = [f"feature_{i}" for i in range(len(importances))]
        
#         # Sort feature importances
#         indices = np.argsort(importances)[::-1]
        
#         # Plot feature importances
#         plt.figure(figsize=(12, 8))
#         plt.title('Feature Importances')
#         plt.bar(range(len(indices)), importances[indices], align='center')
#         plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
#         plt.tight_layout()
#         plt.savefig("feature_importances.png")
#         plt.close()
        
#         # Return top 10 features
#         top_features = [(feature_names[i], importances[i]) for i in indices[:10]]
#         return top_features
#     else:
#         print("The model doesn't support feature importances.")
#         return None

# def create_prediction_function(model):
#     def predict_flood_risk(data_point):
#         """
#         Predict flood risk for a single data point.
        
#         Parameters:
#         - data_point: Dictionary with all required features
        
#         Returns:
#         - Flood probability and binary prediction
#         """
#         # Convert to DataFrame
#         df = pd.DataFrame([data_point])
        
#         # Make prediction
#         proba = model.predict_proba(df)[0, 1]
#         prediction = 1 if proba >= 0.5 else 0
        
#         risk_level = "Low" if proba < 0.3 else "Medium" if proba < 0.7 else "High"
        
#         return {
#             "probability": proba,
#             "prediction": prediction,
#             "risk_level": risk_level
#         }
    
#     return predict_flood_risk

# # Main execution
# def main():
#     # Load and explore data
#     print("Loading and exploring data...")
#     perform_eda(data)
    
#     # Preprocess data
#     print("\nPreprocessing data...")
#     X_train, X_test, y_train, y_test, preprocessor, numerical_cols, categorical_cols = preprocess_data(data)
    
#     # Build models
#     print("\nBuilding models...")
#     models = build_models(X_train, y_train, preprocessor)
    
#     # Evaluate models
#     print("\nEvaluating models...")
#     results = evaluate_models(models, X_train, y_train, X_test, y_test)
    
#     # Find the best model based on ROC-AUC
#     best_model_name = max(results, key=lambda k: results[k]['roc_auc'])
#     print(f"\nBest model: {best_model_name} with ROC-AUC: {results[best_model_name]['roc_auc']:.4f}")
    
#     # Fine-tune the best model
#     best_model = fine_tune_best_model(best_model_name, models, X_train, y_train, X_test, y_test)
    
#     # Get feature importance
#     top_features = feature_importance(best_model, numerical_cols, categorical_cols)
#     if top_features:
#         print("\nTop 10 features:")
#         for feature, importance in top_features:
#             print(f"{feature}: {importance:.4f}")
    
#     # Create prediction function
#     predict_flood_risk = create_prediction_function(best_model)
    
#     # Example prediction
#     example = {
#         'region': 'Region_A',
#         'rainfall_24h': 85.0,
#         'rainfall_72h': 170.0,
#         'river_level': 2.8,
#         'soil_moisture': 75.0,
#         'slope_gradient': 3.5,
#         'vegetation_cover': 60.0,
#         'urbanization': 70.0,
#         'temperature': 22.0,
#         'snowmelt': 5.0,
#         'reservoir_level': 85.0,
#         'previous_floods': 0,
#         'year': 2023,
#         'month': 6,
#         'day': 15
#     }
    
#     prediction = predict_flood_risk(example)
#     print("\nExample prediction:")
#     print(f"Flood probability: {prediction['probability']:.2f}")
#     print(f"Prediction: {'Flood' if prediction['prediction'] == 1 else 'No Flood'}")
#     print(f"Risk level: {prediction['risk_level']}")
    
#     print("\nFlood prediction model successfully built and saved as 'flood_prediction_model.pkl'")

# if __name__ == "__main__":
#     main()

Loading and exploring data...
Dataset shape: (3000, 18)

Data types:
date                 datetime64[ns]
region                       object
rainfall_24h                float64
rainfall_72h                float64
river_level                 float64
soil_moisture               float64
slope_gradient              float64
vegetation_cover            float64
urbanization                float64
temperature                 float64
snowmelt                    float64
reservoir_level             float64
previous_floods               int64
flood_occurred                int64
year                          int64
month                         int64
day                           int64
flood_probability           float64
dtype: object

Check for missing values:
date                 0
region               0
rainfall_24h         0
rainfall_72h         0
river_level          0
soil_moisture        0
slope_gradient       0
vegetation_cover     0
urbanization         0
temperature          0
snowmelt    

d:\anant\hck\cld\my_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:57:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\cld\my_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:57:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\cld\my_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:57:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\cld\my_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:57:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain,

XGBoost Cross-Validation ROC-AUC: 0.9455 ± 0.0096

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       326
           1       0.93      0.85      0.89       274

    accuracy                           0.90       600
   macro avg       0.91      0.90      0.90       600
weighted avg       0.90      0.90      0.90       600


Training Logistic Regression...
Logistic Regression Cross-Validation ROC-AUC: 0.9396 ± 0.0082

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       326
           1       0.90      0.85      0.87       274

    accuracy                           0.89       600
   macro avg       0.89      0.88      0.89       600
weighted avg       0.89      0.89      0.89       600


Best model: Gradient Boosting with ROC-AUC: 0.9588

Fine-tuning Gradient Boosting...
Best parameters: {'classifier__learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib

# Load the dataset
dataset_path = "flood_prediction_dataset.csv"
data = pd.read_csv(dataset_path)

# Convert date to datetime
data['date'] = pd.to_datetime(data['date'])

# Drop the 'region' column
data = data.drop(columns=['region'])

# Preprocess data
def preprocess_data(data):
    X = data.drop(['flood_occurred', 'flood_probability', 'date'], axis=1)
    y = data['flood_occurred']
    numerical_cols = X.columns.tolist()
    preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numerical_cols)])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return X_train, X_test, y_train, y_test, preprocessor, numerical_cols

X_train, X_test, y_train, y_test, preprocessor, numerical_cols = preprocess_data(data)

# Build models
def build_models():
    models = {
        'Random Forest': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        'Gradient Boosting': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'XGBoost': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        'Logistic Regression': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(random_state=42, max_iter=1000))
        ])
    }
    return models

models = build_models()

def evaluate_models(models, X_train, y_train, X_test, y_test):
    results = {}
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        print(f"\n{name} Classification Report:")
        print(classification_report(y_test, y_pred))
        results[name] = model
    return results

results = evaluate_models(models, X_train, y_train, X_test, y_test)

# Save the best model
best_model_name = max(results, key=lambda k: cross_val_score(results[k], X_train, y_train, cv=5, scoring='roc_auc').mean())
joblib.dump(results[best_model_name], "flood_prediction_model.pkl")
print(f"Best model '{best_model_name}' saved as 'flood_prediction_model.pkl'")



Training Random Forest...

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       326
           1       0.93      0.86      0.90       274

    accuracy                           0.91       600
   macro avg       0.91      0.90      0.91       600
weighted avg       0.91      0.91      0.91       600


Training Gradient Boosting...

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       326
           1       0.91      0.86      0.89       274

    accuracy                           0.90       600
   macro avg       0.90      0.90      0.90       600
weighted avg       0.90      0.90      0.90       600


Training XGBoost...

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       326
           1       0.93      0.84      0.89   

d:\anant\hck\main\cld\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\main\cld\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\main\cld\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\anant\hck\main\cld\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:05:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bs

Best model 'Gradient Boosting' saved as 'flood_prediction_model.pkl'


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib

# Load the dataset
dataset_path = "flood_prediction_dataset.csv"
data = pd.read_csv(dataset_path)

# Convert date to datetime
data['date'] = pd.to_datetime(data['date'])

# Preprocess data
def preprocess_data(data):
    # Separate features and target
    X = data.drop(['flood_occurred', 'flood_probability', 'date'], axis=1)
    
    # Remove unwanted features
    X = X[['rainfall_24h', 'rainfall_72h', 'river_level', 'soil_moisture', 'reservoir_level', 'previous_floods', 'temperature']]
    
    y = data['flood_occurred']
    
    # Create preprocessing steps
    numerical_transformer = StandardScaler()
    
    # Create column transformer
    preprocessor = ColumnTransformer(
        transformers=[('num', numerical_transformer, X.columns)])
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    return X_train, X_test, y_train, y_test, preprocessor, X.columns

# Main execution
def main():
    print("\nPreprocessing data...")
    X_train, X_test, y_train, y_test, preprocessor, feature_cols = preprocess_data(data)
    
    # Build and evaluate models
    models = {
        'Random Forest': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        'Gradient Boosting': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'XGBoost': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        'Logistic Regression': Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(random_state=42, max_iter=1000))
        ])
    }
    
    best_model = None
    best_roc_auc = 0
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        
        print(f"{name} ROC-AUC: {roc_auc:.4f}")
        
        if roc_auc > best_roc_auc:
            best_roc_auc = roc_auc
            best_model = model
    
    print("\nSaving best model...")
    joblib.dump(best_model, "flood_prediction_model.pkl")
    print("Model saved as 'flood_prediction_model.pkl'")
    
if __name__ == "__main__":
    main()



Preprocessing data...

Training Random Forest...
Random Forest ROC-AUC: 0.9226

Training Gradient Boosting...
Gradient Boosting ROC-AUC: 0.9351

Training XGBoost...
XGBoost ROC-AUC: 0.9109

Training Logistic Regression...
Logistic Regression ROC-AUC: 0.9356

Saving best model...
Model saved as 'flood_prediction_model.pkl'


d:\anant\hck\main\cld\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:49:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
